In [125]:
from basketball import *

import numpy as np
import pickle
import pandas as pd

## Get Salary Cap info

In [130]:
url = 'https://www.basketball-reference.com/contracts/salary-cap-history.html'

tables = get_tables(url)
df_caps = tables[0]

df_caps.columns = ['Year', 'Salary Cap', 'Salary Cap (2015)']
df_caps.drop(0, inplace=True)
df_caps.reset_index(inplace=True, drop=True)

# Money to float
df_caps['Salary Cap'].replace('\D', '', regex=True, inplace=True)
df_caps['Salary Cap (2015)'].replace('\D', '', regex=True, inplace=True)

df_caps['Salary Cap'] = pd.to_numeric(df_caps['Salary Cap'])
df_caps['Salary Cap (2015)'] = pd.to_numeric(df_caps['Salary Cap (2015)'])

df_caps['Year'] = df_caps['Year'].apply(lambda x: x[:2] + x[-2:])

In [131]:
with open('df_caps.pickle', 'wb') as f:
    pickle.dump(df_caps, f)

In [132]:
df_caps.head()

,Year,Salary Cap,Salary Cap (2015)
0,1985,3600000,7934034.0
1,1986,4233000,9153509.0
2,1987,4945000,10317292.0
3,1988,6164000,12354015.0
4,1989,7232000,13829137.0


## Get Stats

In [121]:
# Advanced tables
with open('databases.pickle', 'rb') as f:
    databases = pickle.load(f)


yearly_dfs = []

for year in databases.keys():
    if year != 2015:
        adv_stats, adv_sals = database_to_stats_and_salaries(pos=8, year=year, database=databases[year])
        per36_stats, per_sals = database_to_stats_and_salaries(pos=6, year=year, database=databases[year])

        df_adv = stats_salary_join(year=year, dfs=adv_stats, targets=adv_sals)
        df_per_36 = stats_salary_join(year=year, dfs=per36_stats, targets=per_sals)
        
        # drop duplicate columns
        cols_to_drop = {'Salary', 'MP', 'G', 'Age'}
        cols_to_keep = list(set(df_per_36.columns) - cols_to_drop)
        df_per_36 = df_per_36[cols_to_keep]
        
        # concat and append the normal stats and advanced stats
        yearly_dfs.append(pd.concat([df_per_36, df_adv], join='inner', axis=1))
        
df_stats = pd.concat(yearly_dfs)
df_stats.head()

,,,TRB,FT%,FT,3P%,FG,FTA,DRB,GS,STL,PTS,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Salary
Name,Team,Year,,,,,,,,,,,,,,,,,,,,,
Paul Pierce,BOS,2008,5.1,0.843,5.1,0.392,6.4,6.1,4.5,80,1.3,19.7,...,6.7,5.7,12.4,0.207,NaN,3.3,1.5,4.7,4.9,16360094
Ray Allen,BOS,2008,3.7,0.907,2.9,0.398,6.0,3.3,2.6,73,0.9,17.5,...,5.6,4.1,9.7,0.177,NaN,3.2,-0.1,3.1,3.4,16000000
Kevin Garnett,BOS,2008,10.1,0.801,4.2,0.000,8.3,5.2,8.0,71,1.5,20.7,...,6.6,6.2,12.9,0.265,NaN,2.7,4.7,7.4,5.5,23750000
Rajon Rondo,BOS,2008,5.0,0.611,1.7,0.263,5.5,2.7,3.8,77,2.0,12.7,...,2.3,4.9,7.2,0.150,NaN,-0.1,2.7,2.6,2.7,1229280
Kendrick Perkins,BOS,2008,8.9,0.623,2.1,0.000,4.0,3.4,6.2,78,0.6,10.2,...,1.9,4.3,6.2,0.156,NaN,-1.7,4.3,2.6,2.2,4480912


## Joining Stats and Caps

In [123]:
# set index to join (year)
df_caps.set_index('Year', inplace=True)
poc = [df_stats.loc[ix]['Salary'] / df_caps.loc[str(ix[2]), 'Salary Cap'] for ix in df_stats.index]
df_stats['Percentage of Cap'] = poc


In [124]:
df_stats.head()

,,,TRB,FT%,FT,3P%,FG,FTA,DRB,GS,STL,PTS,...,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Salary,Percentage of Cap
Name,Team,Year,,,,,,,,,,,,,,,,,,,,,
Paul Pierce,BOS,2008,5.1,0.843,5.1,0.392,6.4,6.1,4.5,80,1.3,19.7,...,5.7,12.4,0.207,NaN,3.3,1.5,4.7,4.9,16360094,0.294088
Ray Allen,BOS,2008,3.7,0.907,2.9,0.398,6.0,3.3,2.6,73,0.9,17.5,...,4.1,9.7,0.177,NaN,3.2,-0.1,3.1,3.4,16000000,0.287615
Kevin Garnett,BOS,2008,10.1,0.801,4.2,0.000,8.3,5.2,8.0,71,1.5,20.7,...,6.2,12.9,0.265,NaN,2.7,4.7,7.4,5.5,23750000,0.426928
Rajon Rondo,BOS,2008,5.0,0.611,1.7,0.263,5.5,2.7,3.8,77,2.0,12.7,...,4.9,7.2,0.150,NaN,-0.1,2.7,2.6,2.7,1229280,0.022097
Kendrick Perkins,BOS,2008,8.9,0.623,2.1,0.000,4.0,3.4,6.2,78,0.6,10.2,...,4.3,6.2,0.156,NaN,-1.7,4.3,2.6,2.2,4480912,0.080548


succesfully integrated percentage of cap into stats df

## Feature Engineering

In [133]:
def prepare_dataframe(df, features):
    df = df.copy()

    # let's drop players with low minutes played per game, say less than 10
    df['MPperG'] = df['MP'] / df['G']
    
    # drop blank columns
    df.drop('\xa0', inplace=True, axis=1)

    # We want over 10 minutes played per game and nonzero salary
    mask = ((df['MPperG'] > 10) & (df['Salary'] > 0))
    df = df[mask]

    X = df[features].values
    y = df['Percentage of Cap'].values
    
    return X, y, df

In [134]:

df.head()

,,,TRB,FT%,FT,3P%,FG,FTA,DRB,GS,STL,PTS,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Salary,Percentage of Cap,MPperG
Name,Team,Year,,,,,,,,,,,,,,,,,,,,,
Paul Pierce,BOS,2008,5.1,0.843,5.1,0.392,6.4,6.1,4.5,80,1.3,19.7,...,5.7,12.4,0.207,3.3,1.5,4.7,4.9,16360094,0.294088,35.925000
Ray Allen,BOS,2008,3.7,0.907,2.9,0.398,6.0,3.3,2.6,73,0.9,17.5,...,4.1,9.7,0.177,3.2,-0.1,3.1,3.4,16000000,0.287615,35.945205
Kevin Garnett,BOS,2008,10.1,0.801,4.2,0.000,8.3,5.2,8.0,71,1.5,20.7,...,6.2,12.9,0.265,2.7,4.7,7.4,5.5,23750000,0.426928,32.788732
Rajon Rondo,BOS,2008,5.0,0.611,1.7,0.263,5.5,2.7,3.8,77,2.0,12.7,...,4.9,7.2,0.150,-0.1,2.7,2.6,2.7,1229280,0.022097,29.948052
Kendrick Perkins,BOS,2008,8.9,0.623,2.1,0.000,4.0,3.4,6.2,78,0.6,10.2,...,4.3,6.2,0.156,-1.7,4.3,2.6,2.2,4480912,0.080548,24.512821


In [138]:
len(y)

3732

## Modelling

In [135]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

### Advanced Features

In [167]:
def run_linear_model(features, scale=False):
    X, y, df = prepare_dataframe(df_stats, features)

    X_cv, X_test, y_cv, y_test = train_test_split(X, y, test_size=.2,
                                                       random_state = 1999)

    kf = KFold(n_splits=5, shuffle=True, random_state=1999)

    train_scores = []
    test_scores = []

    for train_index, val_index in kf.split(X_cv):

        X_train, X_val = X_cv[train_index], X_cv[val_index]
        y_train, y_val = y_cv[train_index], y_cv[val_index]
        
        if scale:
            std = StandardScaler()
            X_train = std.fit_transform(X_train)
            X_val = std.transform(X_val)

        lm = LinearRegression()
        lm.fit(X_train, y_train)

        y_train_pred = lm.predict(X_train)
        y_val_pred = lm.predict(X_val)

        train_scores.append(r2_score(y_train, y_train_pred))
        test_scores.append(r2_score(y_val, y_val_pred))
        
    lm.fit(X_cv, y_cv)
    coefficients = sorted(list(zip(features, lm.coef_)), key=lambda x: -np.abs(x[1]))
    
    print('Training Scores:\t', train_scores)
    print('Test Scores:\t\t', test_scores)
        
    return train_scores, test_scores, coefficients, df 

In [181]:
adv_features = ['Age','PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'WS/48', 'BPM', 'VORP']

train_scores, test_scores, coeffs, df = run_linear_model(adv_features, scale=True)

Training Scores:	 [0.4795631309191546, 0.4731353864330483, 0.45727207199476894, 0.47216308966290665, 0.4716430772463598]
Test Scores:		 [0.41365660170107743, 0.45275630273934064, 0.5082774223947935, 0.4552399596895744, 0.45897318520425745]


In [182]:
coeffs

[('WS/48', -0.528332213935527),
 ('TS%', -0.3674551431850964),
 ('3PAr', -0.05287488381531982),
 ('FTr', 0.031617315255967025),
 ('STL%', -0.02758921531955814),
 ('TRB%', 0.023678450531237422),
 ('ORB%', -0.01789379352453641),
 ('PER', 0.013599729288205192),
 ('DRB%', -0.011113030136457785),
 ('VORP', 0.010549954077677596),
 ('BPM', 0.009232231712099248),
 ('Age', 0.0062619625894862726),
 ('BLK%', -0.006256129864612218),
 ('AST%', -0.002364172138644796),
 ('TOV%', 0.002311614069680533)]

In [184]:
np.mean(test_scores)

0.45778069434580865

### Feature Selection with Lasso

In [205]:
df_stats.columns

Index(['TRB', 'FT%', 'FT', '3P%', 'FG', 'FTA', 'DRB', 'GS', 'STL', 'PTS',
       '2P%', '3P', '2PA', '2P', 'BLK', '3PA', 'AST', 'FGA', 'FG%', 'Rk',
       'ORB', 'PF', 'TOV', 'Rk', 'Age', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', ' ',
       'OWS', 'DWS', 'WS', 'WS/48', ' ', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Salary', 'Percentage of Cap'],
      dtype='object', name=0)

In [214]:
features = ['TRB', 'FT', 'FG', 'FTA', 'DRB', 'STL', 'PTS',
       '2P%', '3P', '2PA', '2P', 'BLK', '3PA', 'AST', 'FGA', 'FG%', 
       'ORB', 'PF', 'TOV', 'Age', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

X, y, df = prepare_dataframe(df_stats, features)

X_cv, X_test, y_cv, y_test = train_test_split(X, y, test_size=.2,
                                                       random_state = 1999)

In [211]:
# examine dataframe for nans

# df.isna().sum()

In [215]:
from sklearn.model_selection import GridSearchCV

lass_pipe = Pipeline([
    ('std_scl', StandardScaler()),
    ('lass', Lasso())
])

param_grid = {
    'lass__alpha': [.001, .01, .1, 1, 10]
}

gcv = GridSearchCV(lass_pipe, param_grid, cv=5)
gcv.fit(X_cv, y_cv)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('std_scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lass', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'lass__alpha': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [216]:
coefficients = sorted(list(zip(features, gcv.best_estimator_.named_steps['lass'].coef_)), key=lambda x: -np.abs(x[1]))
coefficients

[('Age', 0.025202521401964714),
 ('VORP', 0.01661087168283929),
 ('2PA', 0.015085552640407535),
 ('BPM', 0.013426769088038647),
 ('PF', -0.012574586309815628),
 ('USG%', 0.009035117016691805),
 ('DRB%', 0.008494112207228063),
 ('STL', -0.008060425053359528),
 ('FTA', 0.006285482057753487),
 ('TOV%', 0.005248883639453836),
 ('TS%', -0.003602256083461905),
 ('ORB', -0.0035592394204430806),
 ('AST', -0.002504729778789113),
 ('WS/48', -0.0010859498213744486),
 ('BLK', 0.0005236545454175383),
 ('TRB', 0.0),
 ('FT', 0.0),
 ('FG', 0.0),
 ('DRB', 0.0),
 ('PTS', 0.0),
 ('2P%', -0.0),
 ('3P', -0.0),
 ('2P', 0.0),
 ('3PA', -0.0),
 ('FGA', 0.0),
 ('FG%', -0.0),
 ('TOV', 0.0),
 ('PER', 0.0),
 ('3PAr', -0.0),
 ('FTr', 0.0),
 ('ORB%', -0.0),
 ('TRB%', 0.0),
 ('AST%', -0.0),
 ('STL%', -0.0),
 ('BLK%', 0.0),
 ('OBPM', 0.0),
 ('DBPM', 0.0)]

### Last Coefficients left standing

In [217]:
best_tuples = [('Age', 0.025202521401964714),
('VORP', 0.01661087168283929),
('2PA', 0.015085552640407535),
('BPM', 0.013426769088038647),
('PF', -0.012574586309815628),
('USG%', 0.009035117016691805),
('DRB%', 0.008494112207228063),
('STL', -0.008060425053359528),
('FTA', 0.006285482057753487),
('TOV%', 0.005248883639453836),
('TS%', -0.003602256083461905),
('ORB', -0.0035592394204430806),
('AST', -0.002504729778789113),
('WS/48', -0.0010859498213744486),
('BLK', 0.0005236545454175383)]

best_coefs = [t[0] for t in best_tuples]
best_coefs

['Age',
 'VORP',
 '2PA',
 'BPM',
 'PF',
 'USG%',
 'DRB%',
 'STL',
 'FTA',
 'TOV%',
 'TS%',
 'ORB',
 'AST',
 'WS/48',
 'BLK']

In [223]:
tr_scores, te_scores, coeffs, df_lin = run_linear_model(best_coefs)

Training Scores:	 [0.4828263188401576, 0.47809910831279623, 0.4635515325466798, 0.47786102787946605, 0.4739661087908694]
Test Scores:		 [0.4241562577487701, 0.4561118473772212, 0.5078330258921343, 0.45527337578249427, 0.4724347978836221]


In [224]:
coeffs

[('WS/48', -0.12342554930333312),
 ('TS%', -0.12109673705594054),
 ('STL', -0.025987767968061255),
 ('PF', -0.011441998245158956),
 ('VORP', 0.009749568732826383),
 ('BPM', 0.008938468032128069),
 ('ORB', -0.008364724208158451),
 ('Age', 0.006251623718098149),
 ('2PA', 0.00605454708664834),
 ('FTA', 0.0051902935970311054),
 ('AST', -0.004377420992437752),
 ('TOV%', 0.0021342512723124146),
 ('DRB%', 0.00160226616323277),
 ('USG%', 0.001234194404564147),
 ('BLK', -0.0004748950255527909)]

### Running on full data

Index(['TRB', 'FT%', 'FT', '3P%', 'FG', 'FTA', 'DRB', 'GS', 'STL', 'PTS',
       '2P%', '3P', '2PA', '2P', 'BLK', '3PA', 'AST', 'FGA', 'FG%', 'Rk',
       'ORB', 'PF', 'TOV', 'Rk', 'Age', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Salary',
       'Percentage of Cap', 'MPperG'],
      dtype='object', name=0)